# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as sqal
from sklearn.model_selection import train_test_split

In [ ]:
X_columns = 'message'
cat_col_names = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [ ]:
# load data from database
engine = sqal.create_engine('sqlite:///DisasterResponse.db')
# inspector = sqal.inspect(engine)
# inspector.get_schema_names()  # -> ['main']
# inspector.get_table_names(schema='main')  # -> ['Message']
# inspector.get_columns('Message', schema='main')   # -> the goods

df = pd.read_sql_table('Message', engine)

In [ ]:
# correct outlying values in `related` column
df['related'] = np.clip(df['related'], 0, 1)

In [ ]:
text = df[X_columns].values
y = df[cat_col_names].values

In [ ]:
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.33, random_state=42)

### 2. Write a tokenization function to process your text data

In [ ]:
import spacy
en_nlp = spacy.load('en')

In [ ]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [ ]:
# https://realpython.com/natural-language-processing-spacy-python/ was helpful here
def tokenize(text):
    doc = en_nlp(text)
    lemmas = [token.lemma_ for token in doc if token not in stopwords and not token.is_punct]
    stems = [stemmer.stem(lemma) for lemma in lemmas]
    return stems

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [ ]:
pipe = make_pipeline(TfidfVectorizer(tokenizer=tokenize, min_df=5), 
                    MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
%%time
pipe.fit(text_train, y_train)

In [ ]:
# compute rigged score
pipe.score(text_train, y_train)

In [ ]:
# compute real score
pipe.score(text_test, y_test)

Massively overfit with a test value of only 0.278, but a training value of 0.991.

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
%%time
y_pred = pipe.predict(text_test)

In [ ]:
# Supposed to do this column by columm
print(classification_report(y_test, y_pred, target_names=cat_col_names, zero_division=1))

In [ ]:
y_test.shape

In [ ]:
for i in range(y_test.shape[1]):
    print(cat_col_names[i])
    print(classification_report(y_test[:,i], y_pred[:,i], zero_division=1))

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
print("{0:>20}".format(''), 'prec', 'recall', 'f1', sep='\t')
for i in range(y_test.shape[1]):
    print("{0:>20}".format(cat_col_names[i]), '%.2f' % precision_score(y_test[:,i], y_pred[:,i]), '%.2f' % recall_score(y_test[:,i], y_pred[:,i]), '%.2f' % f1_score(y_test[:,i], y_pred[:,i]), sep = '\t')

https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics

### 6. Improve your model

####  Let's split the `Vectorizer` out of the pipeline, precompute it on the training and test sets, and write the results to disk. 
Parameter searching is a slow business and we want to speed things up as much as we can.

In [ ]:
# vect = TfidfVectorizer(tokenizer=tokenize, min_df=5)

In [ ]:
# %%time
# # Pre-calculate X_train and X_test for testing the categorization pipeline - takes 5 minutes to calculate
# X_train1 = vect.fit_transform(text_train)
# X_test1 = vect.transform(text_test)

In [ ]:
# X_train1.shape

In [ ]:
# https://stackoverflow.com/questions/52444921/save-numpy-array-using-pickle
import pickle as pkl

In [ ]:
# fileObject = open('severedPipelineXtrain.pkl', 'wb')
# pkl.dump(X_train1, fileObject)
# fileObject.close()

# fileObject = open('severedPipelineXtest.pkl', 'wb')
# pkl.dump(X_test1, fileObject)
# fileObject.close()

In [ ]:
fileObject = open('severedPipelineXtrain.pkl', 'rb')
X_train = pkl.load(fileObject)
fileObject.close()

fileObject = open('severedPipelineXtest.pkl', 'rb')
X_test = pkl.load(fileObject)
fileObject.close()

In [ ]:
type(X_train)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

#### Alright.  Back to work.  Let's actually evaluate this classifier-only pipeline.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
pipe = make_pipeline(MultiOutputClassifier(estimator=
#                                           RandomForestClassifier(n_jobs=-1)))
#                                           AdaBoostClassifier()))
#                                           LogisticRegression(C=3, max_iter=30, warm_start=True, n_jobs=-1)))
                                            KNeighborsClassifier(n_jobs=-1)))

LR doesn't work here because of the empty 'child_alone' feature.

In [ ]:
%%time
pipe.fit(X_train, y_train)

In [ ]:
print(pipe.score(X_test, y_test))
print(pipe.score(X_train, y_train))

At least it's not overfitting with `AdaBoostClassifier`, like it does with `RandomForestClassifier` (0.256, 0.995).

With KNeighbors we get (0.233, 0.264) but it took 16 minutes to compute the two scores; but less than a second to fit.

In [ ]:
%%time
y_pred = pipe.predict(X_test)

Took 5 mins with KNeighbors.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred, target_names=cat_col_names, zero_division=1))

#### Let's do some parameter searching...

Use grid search to find better parameters. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
pipe = make_pipeline(MultiOutputClassifier(estimator=
#                                           RandomForestClassifier(n_jobs=-1)))  # This takes almost 11 minutes
                                           AdaBoostClassifier(base_estimator=
                                                              DecisionTreeClassifier(max_depth=1),
                                               n_estimators=10, learning_rate=1))) # 1 minute
#                                           LogisticRegression(C=3, max_iter=30, warm_start=True))) 
#                                                              solver='saga', penalty='elasticnet')))
                                                              # class_weight='balanced')))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# param_grid = {}
# param_grid = {'multioutputclassifier__estimator__n_estimators': [10, 30, 100]} 
# param_grid = {'multioutputclassifier__estimator__learning_rate': [0.3, 1.0, 3.0]}
# param_grid = {'multioutputclassifier__estimator__C': [3.0, 10.0, 30.0]}
# param_grid = {'multioutputclassifier__estimator__l1_ratio': [0.1, 0.3, 0.7, 1.0]}
#param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_split': [2, 4, 8, 16]}
param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': [1, 2, 4, 8]}

In [ ]:
grid = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, n_jobs=-1)

In [ ]:
grid.estimator['multioutputclassifier'].estimator.base_estimator

In [ ]:
%%time
grid.fit(X_train, y_train)

25min for miniscule benefit - LR with saga not worth pursuing.  (Although the highest score fit (l1_ratio = 0.7) was only 1/3 the duration of the most expensive.  Could be because of the warm_start setting.)

For the DecisionTreeClassifier, the default value of `min_samples_split` of 2 is the best, but the effect is very weak. [2,4,8,16]

Equilvalently, for DTC, the default value of `min_samples_leaf` was best, and varying it made no difference. [1,2,4,8]

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
grid.cv_results_

In [ ]:
grid.estimator.named_steps['multioutputclassifier'].estimator.base_estimator

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.